In [1]:
!conda activate DS807

In [2]:
import tensorflow as tf
import data_loader as dl
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt

In [3]:
df = dl.load_1d_grays()
df

,index,string,CC,D,Y,gray_value,filename
0,1,1836,0,3,6,"[[[166.0], [167.0], [166.0], [162.0], [164.0],...",1.jpg
1,2,1836,0,3,6,"[[[169.0], [168.0], [169.0], [170.0], [166.0],...",2.jpg
2,3,1840,0,4,0,"[[[167.0], [169.0], [169.0], [166.0], [165.0],...",3.jpg
3,4,1840,0,4,0,"[[[167.0], [165.0], [165.0], [165.0], [163.0],...",4.jpg
4,5,1823,0,2,3,"[[[62.0], [45.0], [45.0], [37.0], [40.0], [33....",5.jpg
...,...,...,...,...,...,...,...
11995,11996,1808,0,0,8,"[[[88.0], [62.0], [60.0], [68.0], [61.0], [101...",11996.jpg
11996,11997,1809,0,0,9,"[[[173.0], [171.0], [173.0], [175.0], [174.0],...",11997.jpg
11997,11998,1840,0,4,0,"[[[189.0], [189.0], [189.0], [189.0], [189.0],...",11998.jpg
11998,11999,1840,0,4,0,"[[[191.0], [192.0], [192.0], [192.0], [193.0],...",11999.jpg


In [4]:
arr = np.zeros(shape=(12000,56,106,1))
i = 0
while i < 12000:
    arr[i] = df['gray_value'][i]
    i += 1
arr.shape

(12000, 56, 106, 1)

In [5]:
x_values = arr/255
cc_values = df['CC'].astype(np.uint8)
d_values = df['D'].astype(np.uint8)
y_values = df['Y'].astype(np.uint8)

In [6]:
from sklearn.model_selection import train_test_split
#D
xD_train, xD_test, yD_train, yD_test = train_test_split(x_values, d_values, test_size=0.2, random_state=2022)
xD_train, xD_val, yD_train, yD_val = train_test_split(xD_train, yD_train, test_size=0.2, random_state=2022)
print(xD_train.shape, xD_val.shape, xD_test.shape)

(7680, 56, 106, 1) (1920, 56, 106, 1) (2400, 56, 106, 1)


In [11]:
models = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(56, 106, 1)),
    #tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(), # flatten before fully connected part
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(11, activation='softmax'),
    ])

models.compile(
    optimizer='adam',
    loss= 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
    )
#models.summary()

In [ ]:
#seed(100) #set fixed random seed from numpy
history = models.fit(xD_train, yD_train, validation_data=(xD_val, yD_val), epochs=30, verbose=0)

plt.plot(history.history['accuracy'], label = 'Train')
plt.plot(history.history['val_accuracy'], label = 'Val')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()